In [1]:
import importlib.util, sys
print(importlib.util.find_spec("madminer"))

ModuleSpec(name='madminer', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f5b99d9f640>, origin='/home/hep/jl3222/projects/DiHiggs_SBI_4b/madminer/madminer/__init__.py', submodule_search_locations=['/home/hep/jl3222/projects/DiHiggs_SBI_4b/madminer/madminer'])


In [2]:
import madminer
import madminer.utils.morphing as morphing

print("MadMiner path:", madminer.__file__)
print("Available in morphing:", [n for n in dir(morphing) if "plot" in n or "morph" in n])

MadMiner path: /home/hep/jl3222/projects/DiHiggs_SBI_4b/madminer/madminer/__init__.py
Available in morphing: []


In [3]:
import logging
from madminer import MadMiner
#from madminer.utils.morphing import (
#    plot_nd_morphing_basis_scatter,
#    plot_nd_morphing_basis_slices,
#)


In [4]:
# MadMiner output
logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)

# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [5]:
import yaml
with open("workflow.yaml", "r") as file:
    workflow = yaml.safe_load(file)

We use the SMEFT@NLO (https://feynrules.irmp.ucl.ac.be/wiki/SMEFTatNLO) model

Signal decay: gg -> H( -> b b~) H( -> a a)

For the time being, we'll focus on 3 parameters:

- cp (#5)
- cdp (#4)
- cpDC (#2)

## Input the SMEFT@NLO coefficients

In [6]:
miner = MadMiner()

miner.add_parameter(
    lha_block="DIM6",
    lha_id=5,
    parameter_name="cp",
    morphing_max_power=2,
    param_card_transform="theta",
    parameter_range=(-16, 8),
)


miner.add_parameter(
    lha_block="DIM6",
    lha_id=4,
    parameter_name="cdp",
    morphing_max_power=2,
    param_card_transform="theta",
    parameter_range=(-5, 6),
)

miner.add_parameter(
    lha_block="DIM62F",
    lha_id=19,
    parameter_name="ctp",
    morphing_max_power=2,
    param_card_transform="theta",
    parameter_range=(-6, 8),
)


17:45 madminer.core.madmin INFO    Adding parameter: cp (LHA: DIM6 5, Power: 2, Range: (-16, 8))
17:45 madminer.core.madmin WARNING Resetting benchmarks and morphing
17:45 madminer.core.madmin INFO    Adding parameter: cdp (LHA: DIM6 4, Power: 2, Range: (-5, 6))
17:45 madminer.core.madmin WARNING Resetting benchmarks and morphing
17:45 madminer.core.madmin INFO    Adding parameter: ctp (LHA: DIM62F 19, Power: 2, Range: (-6, 8))
17:45 madminer.core.madmin WARNING Resetting benchmarks and morphing


In [7]:
# Manually add in the SM benchmark

miner.add_benchmark({"cp": 0.0, "cdp": 0.0, "ctp": 0.0}, "sm")

17:45 madminer.core.madmin INFO    Added benchmark sm: cp = 0.00e+00, cdp = 0.00e+00, ctp = 0.00e+00


In [ ]:
# Choose an optimal set of non-SM benchmarks over many trials

miner.set_morphing(include_existing_benchmarks=True,
                   max_overall_power=2,
                   n_trials=10000,
                   n_test_thetas=1000)

17:45 madminer.core.madmin INFO    Optimizing basis for morphing
17:49 madminer.core.madmin INFO    Set up morphing with 3 parameters, 10 morphing components, 1 predefined basis points, and 9 new basis points


In [9]:
fig = plot_nd_morphing_basis_slices(miner.morpher)

NameError: name 'plot_nd_morphing_basis_slices' is not defined

In [10]:
# Save out the benchmarks file

miner.save(workflow["morphing_setup"], )

17:50 madminer.core.madmin INFO    Saving setup (including morphing) to /vols/cms/jl3222/nsbi_photon_final_state/data/setup.h5
